In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!unzip open.zip

Archive:  open.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# LabelEncoder로 SUBCLASS를 숫자로 변환
label_encoder = LabelEncoder()
train_df['SUBCLASS'] = label_encoder.fit_transform(train_df['SUBCLASS'])

# 모든 값이 'WT'인 열을 제거하는 작업
columns_to_drop = [col for col in train_df.columns if (train_df[col] == 'WT').all()]

# 열 제거
train_df_cleaned = train_df.drop(columns=columns_to_drop)
test_df_cleaned = test_df.drop(columns=columns_to_drop)

# X, y 분리
X = train_df_cleaned.drop(columns=['ID', 'SUBCLASS'])
y = train_df_cleaned['SUBCLASS']
X_test = test_df_cleaned.drop(columns=['ID'])

# 범주형 변이 정보를 LabelEncoder로 처리 (학습 데이터만 인코딩)
for col in X.columns:
    if X[col].dtype == 'object':
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col])  # 학습 데이터 인코딩
        le_classes = set(le.classes_)
        X_test[col] = X_test[col].map(lambda x: le.transform([x])[0] if x in le_classes else -1)  # 학습 데이터에 없는 값은 -1로 처리

# 학습 및 검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# LightGBM 데이터셋 생성
d_train = lgb.Dataset(X_train, label=y_train)
d_val = lgb.Dataset(X_val, label=y_val, reference=d_train)

# LightGBM 파라미터 설정
params = {
    'objective': 'multiclass',  # 다중 클래스 분류
    'num_class': len(set(y_train)),  # 클래스 개수
    'metric': 'multi_logloss',
    'learning_rate': 0.1,
    'num_leaves': 31,
    'boosting_type': 'gbdt',
    'verbose': -1,
    'random_state': 42
}

# LightGBM 모델 학습 (콜백으로 조기 종료 처리)
lgb_model = lgb.train(params,
                      d_train,
                      valid_sets=[d_train, d_val],
                      num_boost_round=100,
                      callbacks=[lgb.early_stopping(stopping_rounds=10)])

# 검증 데이터에 대한 예측
y_pred_val = lgb_model.predict(X_val, num_iteration=lgb_model.best_iteration)
y_pred_val_labels = [list(p).index(max(p)) for p in y_pred_val]  # 확률 값을 클래스 레이블로 변환

# 검증 데이터 성능 평가
val_accuracy = accuracy_score(y_val, y_pred_val_labels)
print(f'LightGBM 검증 데이터 정확도: {val_accuracy}')

# 테스트 데이터 예측
y_pred_test = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)
y_pred_test_labels = [list(p).index(max(p)) for p in y_pred_test]

# 정수형으로 예측된 SUBCLASS 값을 다시 원래 클래스 이름으로 변환
y_pred_test_labels = label_encoder.inverse_transform(y_pred_test_labels)

# 제출 파일 생성
submission_df = pd.DataFrame({
    'ID': test_df_cleaned['ID'],
    'SUBCLASS': y_pred_test_labels  # 원래 SUBCLASS 이름으로 출력
})

submission_df.to_csv('submission_lightgbm_fixed.csv', index=False)


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[20]	training's multi_logloss: 0.989999	valid_1's multi_logloss: 2.13369
LightGBM 검증 데이터 정확도: 0.33843674456083805
